**This notebook will try to implement the tensboard plots, along with the hyperparameter optimisation**

In [1]:
import trading_gym
from trading_gym.registry.gaia.v9.env import GAIAPredictorsContinuousV9
from trading_gym.registry.gaia.v8.env import GAIAPredictorsContinuousV8
from datetime import datetime
from trading_gym.contracts import Cash
from collections import namedtuple
import json
import tensorflow as tf
import os
import pandas as pd
import ray
print(datetime.now())
print(trading_gym.__name__, trading_gym.__version__)
print(ray.__name__, ray.__version__)

2019-08-13 08:43:25.008890
trading_gym 0.8.1
ray 0.7.2


In [2]:
import gym
import numpy as np
import stable_baselines
from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines import DDPG

In [4]:
import os
os.getcwd()

'/home/Nicholas/trading-gym/notebooks/trading-gym/walk-forward'

In [5]:
import sys 
sys.path.insert(1,'/home/Nicholas/trading-gym/rl-baselines-zoo/utils')

In [6]:
import utils
# import 

In [7]:
from utils import make_env, ALGOS, linear_schedule, get_latest_run_id, get_wrapper_class
from hyperparams_opt import hyperparam_optimization

In [8]:
import numpy as np
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines import PPO2
from stable_baselines.common.policies import MlpLnLstmPolicy
import optuna

In [11]:
n_cpu = 4


def optimize_PPO(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.4),
        'noptepochs': int(trial.suggest_loguniform('noptepochs', 1, 48)),
        'lam': trial.suggest_uniform('lam', 0.8, 1.)
    }

def optimize_DDPG(trial):
    """ Learning hyperparamters we want to optimise"""
    n_actions = 2
    return {
        # the ones we won't optimise 
        
        'action_noise': OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions),\
                                                     sigma=float(0.5) * np.ones(n_actions)),
        'param_noise': None, # AdaptiveParamNoiseSpec(initial_stddev=0.1, desired_action_stddev=0.1)

        # ...and the ones we will
        'nb_train_steps': int(trial.suggest_loguniform('n_steps', 16, 2048)),
        'gamma': trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'actor_lr': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        'critic_lr': trial.suggest_loguniform('learning_rate', 1e-5, 1.)
    }



def optimize_TD3(trial):
    """ Learning hyperparamters we want to optimise"""
    return {
        # the ones we won't optimise 
        
        
        # ...and the ones wee will
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.),
        
    }
    

In [12]:
# three different specific hyperparams for the agents that we won't change or try to optimize 
td3_hyparam = dict()
td3_hyparam['policy'] = 'MlpPolicy'
td3_hyparam['tensorboard_log'] = "optuna_tensorboard/optuna_td3v9"

sac_hyparam = dict()
sac_hyparam['policy'] = 'MlpPolicy'
sac_hyparam['tensorboard_log'] = "optuna_tensorboard/optuna_sacv9"


ddpg_hyparam = dict()
ddpg_hyparam['policy'] = 'MlpPolicy'
ddpg_hyparam['tensorboard_log'] = "optuna_tensorboard/optuna_DDPGv9"

**IMPORTANT:
One needs to choose the agent to optimize**


In [13]:
# agent = 'sac'
agent = 'ddpg'
# agent = 'td3'

In [16]:
def optimize_agent(trial):
    
    """ Train the model and optimise
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    
    env_config = dict()
    env_config['folds'] =  {
        'training-set': [datetime.min, datetime(2008, 3, 18)],
        'test-set': [datetime(2008, 3, 19), datetime.max],
    }
    
    env = GAIAPredictorsContinuousV9(env_config)
    
    # implement a way of getting MlpLnLstmPolicy in (and other policies)
    if agent == 'sac':
        model_params = optimize_SAC(trial)
        env = SubprocVecEnv([lambda: env for i in range(n_cpu)])
        model = SAC(sac_hyparam['policy'], env, verbose=0, **model_params)
        
        
    if agent == 'ddpg':
        model_params = optimize_DDPG(trial)
        env = DummyVecEnv([lambda: env])
        model = DDPG(ddpg_hyparam['policy'], env, verbose=0, **model_params)
        
        
    if agent == 'td3': 
        model_params = optimize_TD3(trial)
        env = DummyVecEnv([lambda: env])
        model = TD3(td3_hyparam['policy'], env, verbose=0, **model_params)
        
    # note, depending on the model/agent, it may or may not allow for multiple vectorized environments. 
#     env = SubprocVecEnv([lambda: env for i in range(n_cpu)])
    
    # now let us let the model learn 
    print("Model now learning")
    model.learn(10000)
    print("Model finished learning")
    
    rewards = []
    n_episodes, reward_sum = 0, 0.0
    
    print("Now some optimisation")
    obs = env.reset()
    
    # how many different episodes we want to run
    while n_episodes < 4:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward
        
        
        if done:
            rewards.append(reward_sum)
            reward_sum = 0.0
            n_episodes += 1
            obs = env.reset()
            print("Another episode done")

    last_reward = np.mean(rewards)
    trial.report(-1 * last_reward)
    
    print("Finished")
    print(last_reward)
    
    return -1 * last_reward

In [17]:
study = optuna.create_study(study_name='SAC_v9_optuna', load_if_exists=True)
# storage can go for now --  storage='sqlite:///params.db'
study.optimize(optimize_agent, n_trials=1000, n_jobs=4)

Model now learning
Model now learning
Model now learning
Model now learning
0.010627627


[I 2019-07-31 12:58:56,622] Finished trial#3 resulted in value: -0.0106276273727417. Current best value is -0.0106276273727417 with parameters: {'n_steps': 51.784331509268064, 'gamma': 0.8058968754328775, 'learning_rate': 0.13227274109917356}.


0.013625093


[I 2019-07-31 13:04:41,459] Finished trial#1 resulted in value: -0.013625092804431915. Current best value is -0.013625092804431915 with parameters: {'n_steps': 84.51897871253432, 'gamma': 0.9680524288547906, 'learning_rate': 3.116774334972532e-05}.


Model finished learning
Now some optimisation
Finished
0.0075587993


...

[I 2019-08-02 07:52:53,303] Finished trial#205 resulted in value: 0.0021117436699569225. Current best value is -0.03037210926413536 with parameters: {'n_steps': 218.7153107041124, 'gamma': 0.9501558596684897, 'learning_rate': 0.023345300113504476}.


Model now learning
Model finished learning
Now some optimisation
Finished
0.010931166


[I 2019-08-02 07:53:25,828] Finished trial#204 resulted in value: -0.010931165888905525. Current best value is -0.03037210926413536 with parameters: {'n_steps': 218.7153107041124, 'gamma': 0.9501558596684897, 'learning_rate': 0.023345300113504476}.


Model now learning


KeyboardInterrupt: 

In [72]:
env_config = dict()
env_config['folds'] =  {
    'training-set': [datetime.min, datetime(2008, 3, 18)],
    'test-set': [datetime(2008, 3, 19), datetime.max],
}
env = GAIAPredictorsContinuousV9(env_config)

**One should only have to change this cell, and the rest of the experiments will then run from there**

In [73]:
optim_hparam = dict()

optim_hparam['env_id'] = env
# env = DummyVecEnv([lambda: optim_hparam['env_id']])
# env = DummyVecEnv([lambda: env])

optim_hparam['tb_log_dir'] = 'path' #path to the tensoarbod logs

optim_hparam['RL_algo'] = 'ddpg'   #the algorithm one wants to use

optim_hparam['n_trials'] = 100  # number of trials for optimizing hyperparameters 

optim_hparam['n_parallel_jobs'] = 2 # Number of parallel jobs when optimizing hyperparameters

optim_hparam['sampler'] = 'tpe'# Sampler to use when optimizing hyperparameters -- choices['random','tpe']

optim_hparam['pruner'] = 'halving' # Pruner to use when optimizing hyperparamters -- 
# choices['halving','median','none']

optim_hparam['verbose'] = 1 # 0 none, 1 INFO

optim_hparam['log_folder'] = 'path'

optim_hparam['pretrained'] = '' # path to a pretrained agent, if want to continue training

**Now for the actual optimisation**

In [53]:
tensorboard_log = None if optim_hparam['tb_log_dir'] == '' else os.path.join(optim_hparam['tb_log_dir']\
                                                                         ,optim_hparam['RL_algo'] + '_'\
                                                                        ,str(optim_hparam['env_id']))

In [54]:
hyperparams = dict()

n_actions = env.action_space.shape[-1]
hyperparams['noise_std]'] = [float(0.5),float(1.0)]
hyperparams['action_noise'] = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions),\
                                                           sigma=hyperparams['noise_std]'] * np.ones(n_actions))

hyperparams['param_noise'] = None
# AdaptiveParamNoiseSpec(initial_stddev=noise_std,desired_action_stddev=noise_std)

hyperparams['n_timesteps'] = 500000

hyperparams['n_envs'] = 1
# hyperparams['env_wrapper'] = 'GAIAPredictorsContinuousV9' # need to establish my envs wrapper here

# env_wrapper = get_wrapper_class(hyperparams)

In [69]:
def create_env(n_envs): 
    global hyperparams
    
    # look into the wrapper_class documentation 
    env = DummyVecEnv([make_env(optim_hparam['env_id'])])
    env = VecNormalize(env)

    
    return env 

In [65]:
def create_model(*_args,**kwargs):
    '''
    Cretae a model with different hyperparams
    '''
    verbose = 0 
    return optim_hparam['RL_algo'](env=env, tensorboard_log = tensorboard_log,
                                  verbose = verbose, **kwargs)

In [74]:
print("Optimizing hyperparameters")

data_frame = hyperparam_optimization(optim_hparam['RL_algo'],create_model,optim_hparam['env_id'],
                                    n_trials = optim_hparam['n_trials'], 
                                    hyperparams=hyperparams,
                                    n_jobs = optim_hparam['n_parallel_jobs'], 
                                    sampler_method = optim_hparam['sampler'], 
                                    pruner_method = optim_hparam['pruner'], 
                                    verbose = optim_hparam['verbose'])

report_name = "report_{}_{}-trials-{}-{}-{}.csv".format(optim_hparam['env_id'], \
                                                        optim_hparam['n_trials'],\
                                                        optim_hparam['sampler'], \
                                                        optim_hparam['pruner'])

log_path = os.path.join(optim_hparam['log_folder'], optim_hparam['RL_algo'], report_name)

print("Writing report to {}".format(log_path))

os.makedirs(os.path.dirname(log_path), exist_ok=True)
data_frame.to_csv(log_path)
print("Optimization complete")

Optimizing hyperparameters
Sampler: tpe - Pruner: halving


[W 2019-07-30 23:46:53,388] Setting status of trial#0 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:46:53,392] Setting status of trial#1 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_ac

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:00,667] Setting status of trial#15 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:01,666] Setting status of trial#16 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:08,695] Setting status of trial#30 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:08,696] Setting status of trial#31 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:15,726] Setting status of trial#45 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:16,724] Setting status of trial#46 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:23,735] Setting status of trial#60 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:24,129] Setting status of trial#61 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:31,210] Setting status of trial#75 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:31,210] Setting status of trial#76 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:38,215] Setting status of trial#90 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zoo/utils/hyperparams_opt.py", line 76, in objective
    trial.n_actions = env_fn(n_envs=1).action_space.shape[0]
TypeError: 'GAIAPredictorsContinuousV9' object is not callable
[W 2019-07-30 23:47:39,215] Setting status of trial#91 as TrialState.FAIL because of the following error: TypeError("'GAIAPredictorsContinuousV9' object is not callable",)
Traceback (most recent call last):
  File "/home/Nicholas/.venv/lib/python3.6/site-packages/optuna/study.py", line 468, in _run_trial
    result = func(trial)
  File "/home/Nicholas/trading-gym/rl-baselines-zo

Number of finished trials:  100
Best trial:


ValueError: No trials are completed yet.

In [ ]:
# first give it a go with two folds. 




n_actions = env.action_space.shape[-1]
param_noise = None
# param_noise = AdaptiveParamNoiseSpec(initial_stddev=0.1, desired_action_stddev=0.1)
t_steps = 500000

action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

#  if you use param_noise, need to use LnMlpPolicy instead 
model = DDPG(MlpPolicy, env, verbose=0, param_noise=param_noise, action_noise=action_noise)
model.learn(total_timesteps=t_steps)
t_steps = '500k'
model.save("saved_models/2folds_tuning/DDPG/DDPG_v9_steps={}".format(t_steps))

# del model # remove to demonstrate saving and loading

Then reload the agent and  

In [ ]:
episode = env.sample_episode(
        fold = 'test-set',
        stable_model=True,
        model=model,
        episode_length=None,
        benchmark=env._load_benchmark().squeeze(),
        risk_free=env._load_risk_free().squeeze(),
        burn=1,
    )
episode.renderer.plotly_report()
episode.renderer.tearsheet()